<a href="https://colab.research.google.com/github/blinov-89/Tula_CatBoostRegressor/blob/main/Tula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 85 kB/s 


In [3]:
#import необходимых модулей

import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# data

In [42]:
#Считывание данных в DataFrame 

test = pd.read_csv('/content/drive/MyDrive/Тула ИИ/test_dataset_test.csv', sep=';', index_col=None, dtype={'PATIENT_SEX':str, 'MKB_CODE':str, 'ADRES':str, 'VISIT_MONTH_YEAR':str, 'AGE_CATEGORY':str})

In [43]:
train = pd.read_csv('/content/drive/MyDrive/Тула ИИ/train_dataset_train.csv', sep=';', index_col=None, dtype={'PATIENT_SEX':str, 'MKB_CODE':str, 'ADRES':str, 'VISIT_MONTH_YEAR':str, 'AGE_CATEGORY':str, 'PATIENT_ID_COUNT':int})

In [44]:
# train = pd.read_csv('/content/drive/MyDrive/Тула ИИ/train_dataset_train.csv', sep=';')

# анализ

In [45]:
train

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,A00.0,Гурьевск,08.21,young,1
1,0,A00.0,Калининград,03.20,children,1
2,0,A00,Гусев,03.19,children,1
3,0,A00,Калининград,01.22,children,1
4,0,A00,Калининград,02.18,children,1
...,...,...,...,...,...,...
2212388,1,Z99.1,Гурьевск,12.21,children,1
2212389,1,Z99.8,Калининград,10.21,young,1
2212390,1,Z99.9,Калининград,04.19,children,2
2212391,1,Z99.9,Калининград,08.19,children,1


In [46]:
train.describe()

,PATIENT_ID_COUNT
count,2.212393e+06
mean,5.847853e+00
std,6.042780e+01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,2.000000e+00
max,1.353200e+04


In [47]:
train.describe(include=[object])

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY
count,2212393,2212393,2212393,2212393,2212393
unique,2,7644,118,51,6
top,0,J06.9,Калининград,10.19,young
freq,1316709,19100,617129,63145,552363


In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212393 entries, 0 to 2212392
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   PATIENT_SEX       object
 1   MKB_CODE          object
 2   ADRES             object
 3   VISIT_MONTH_YEAR  object
 4   AGE_CATEGORY      object
 5   PATIENT_ID_COUNT  int64 
dtypes: int64(1), object(5)
memory usage: 101.3+ MB


In [49]:
train['PATIENT_ID_COUNT'].value_counts()

1       1366778
2        318988
3        136250
4         77480
5         50492
         ...   
1241          1
4501          1
1166          1
1415          1
6733          1
Name: PATIENT_ID_COUNT, Length: 1688, dtype: int64

In [50]:
train['PATIENT_SEX'].value_counts()

0    1316709
1     895684
Name: PATIENT_SEX, dtype: int64

In [51]:
train['MKB_CODE'].value_counts()

J06.9    19100
I11.9    17282
Z00.0    15419
Z02.7    13458
K02.1    13132
         ...  
D76.1        1
Z44.3        1
D80.7        1
D82.9        1
Z96.2        1
Name: MKB_CODE, Length: 7644, dtype: int64

In [52]:
train['ADRES'].value_counts()

Калининград           617129
Гурьевск              116111
Светлый                85748
Пионерский             84932
Советск                78466
                       ...  
Ясная Поляна            2670
Малинники               2649
Гурьевский              2635
СТ Радуга               2588
СТ Железнодорожник      2517
Name: ADRES, Length: 118, dtype: int64

In [53]:
train['AGE_CATEGORY'].value_counts()

young           552363
elderly         514680
middleage       438426
children        430511
old             214493
centenarians     61920
Name: AGE_CATEGORY, dtype: int64

# **дата**

In [54]:
train

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,A00.0,Гурьевск,08.21,young,1
1,0,A00.0,Калининград,03.20,children,1
2,0,A00,Гусев,03.19,children,1
3,0,A00,Калининград,01.22,children,1
4,0,A00,Калининград,02.18,children,1
...,...,...,...,...,...,...
2212388,1,Z99.1,Гурьевск,12.21,children,1
2212389,1,Z99.8,Калининград,10.21,young,1
2212390,1,Z99.9,Калининград,04.19,children,2
2212391,1,Z99.9,Калининград,08.19,children,1


In [55]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212393 entries, 0 to 2212392
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   PATIENT_SEX       object
 1   MKB_CODE          object
 2   ADRES             object
 3   VISIT_MONTH_YEAR  object
 4   AGE_CATEGORY      object
 5   PATIENT_ID_COUNT  int64 
dtypes: int64(1), object(5)
memory usage: 101.3+ MB


In [56]:
train['day'] = '01.'

In [57]:
train['VISIT_MONTH_YEAR']=train['VISIT_MONTH_YEAR'].astype(str)

In [58]:
train['VISIT_MONTH_YEAR'] = train['day'] +  train['VISIT_MONTH_YEAR']

In [59]:
train['VISIT_MONTH_YEAR'] = pd.to_datetime(train['VISIT_MONTH_YEAR'], dayfirst=True)

In [60]:
train = train.drop('day', axis=1)

In [63]:
pd.to_datetime(train['VISIT_MONTH_YEAR']).dt.month

0           8
1           3
2           3
3           1
4           2
           ..
2212388    12
2212389    10
2212390     4
2212391     8
2212392    11
Name: VISIT_MONTH_YEAR, Length: 2212393, dtype: int64

In [64]:
pd.to_datetime(train['VISIT_MONTH_YEAR']).dt.year.value_counts()

2019    697635
2018    669456
2021    391444
2020    367469
2022     86389
Name: VISIT_MONTH_YEAR, dtype: int64

#test

In [67]:
test

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY
0,0,A00,Калининград,04.22,children
1,0,A00,Калининград,04.22,elderly
2,0,A00,Калининград,04.22,middleage
3,0,A00,Калининград,04.22,young
4,0,A01,Калининград,04.22,middleage
...,...,...,...,...,...
39368,1,Z96.6,Балтийск,04.22,elderly
39369,1,Z96.6,Гусев,04.22,middleage
39370,1,Z96.7,Гусев,04.22,young
39371,1,Z98.8,Озерск,04.22,children


In [68]:
test['VISIT_MONTH_YEAR']

0        04.22
1        04.22
2        04.22
3        04.22
4        04.22
         ...  
39368    04.22
39369    04.22
39370    04.22
39371    04.22
39372    04.22
Name: VISIT_MONTH_YEAR, Length: 39373, dtype: object

In [69]:
test['day'] = '01.'

In [70]:
test['VISIT_MONTH_YEAR']=test['VISIT_MONTH_YEAR'].astype(str)

In [71]:
test['VISIT_MONTH_YEAR'] = test['day'] +  test['VISIT_MONTH_YEAR']

In [72]:
test['VISIT_MONTH_YEAR'] = pd.to_datetime(test['VISIT_MONTH_YEAR'], dayfirst=True)

In [73]:
test = test.drop('day', axis=1)

In [74]:
pd.to_datetime(test['VISIT_MONTH_YEAR']).dt.month.value_counts()

4    39373
Name: VISIT_MONTH_YEAR, dtype: int64

In [75]:
pd.to_datetime(test['VISIT_MONTH_YEAR']).dt.year.value_counts()

2022    39373
Name: VISIT_MONTH_YEAR, dtype: int64

In [76]:
train

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,A00.0,Гурьевск,2021-08-01,young,1
1,0,A00.0,Калининград,2020-03-01,children,1
2,0,A00,Гусев,2019-03-01,children,1
3,0,A00,Калининград,2022-01-01,children,1
4,0,A00,Калининград,2018-02-01,children,1
...,...,...,...,...,...,...
2212388,1,Z99.1,Гурьевск,2021-12-01,children,1
2212389,1,Z99.8,Калининград,2021-10-01,young,1
2212390,1,Z99.9,Калининград,2019-04-01,children,2
2212391,1,Z99.9,Калининград,2019-08-01,children,1


# обучение

In [77]:
#Отделение меток от данных

X = train[['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'VISIT_MONTH_YEAR', 'AGE_CATEGORY']]
y = train[['PATIENT_ID_COUNT']]

In [78]:
#Разделение на train/test для локального тестирования

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [79]:
#Создание объекта данных Pool, плюсы: возможность указать какие признаки являются категориальными

pool_train = Pool(X_train, y_train, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])
pool_test = Pool(X_test, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])

In [80]:
# #Создание объекта данных Pool, плюсы: возможность указать какие признаки являются категориальными

# pool_train = Pool(X_train, y_train, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'VISIT_MONTH_YEAR', 'AGE_CATEGORY'])
# pool_test = Pool(X_test, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'VISIT_MONTH_YEAR', 'AGE_CATEGORY'])

In [89]:
#Объявление CatBoostRegressor и обучение

model = CatBoostRegressor()
model.fit(pool_train)

Learning rate set to 0.133461
0:	learn: 56.1166603	total: 1.13s	remaining: 18m 49s
1:	learn: 52.9101042	total: 1.96s	remaining: 16m 19s
2:	learn: 50.2107538	total: 2.67s	remaining: 14m 48s
3:	learn: 47.8664354	total: 3.21s	remaining: 13m 18s
4:	learn: 45.9906041	total: 3.72s	remaining: 12m 20s
5:	learn: 44.5529763	total: 4.25s	remaining: 11m 44s
6:	learn: 43.2129939	total: 4.76s	remaining: 11m 14s
7:	learn: 42.2868659	total: 5.33s	remaining: 11m
8:	learn: 41.1540986	total: 6s	remaining: 11m
9:	learn: 40.2137767	total: 6.68s	remaining: 11m 1s
10:	learn: 39.4975354	total: 7.35s	remaining: 11m
11:	learn: 38.9536498	total: 7.87s	remaining: 10m 48s
12:	learn: 38.3110027	total: 8.7s	remaining: 11m
13:	learn: 37.9288566	total: 9.26s	remaining: 10m 52s
14:	learn: 37.1390622	total: 9.97s	remaining: 10m 54s
15:	learn: 36.6673458	total: 10.7s	remaining: 10m 55s
16:	learn: 36.2376137	total: 11.7s	remaining: 11m 15s
17:	learn: 35.7237369	total: 12.3s	remaining: 11m 13s
18:	learn: 35.4275872	total: 

In [ ]:
# #Формируем sample_solution. В обучении используется весь train, ответы получаем на test

# pool_train_solution = Pool(X, y, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])
# pool_test_solution = Pool(test, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])

# model_solution = CatBoostRegressor(early_stopping_rounds = 20)
# model_solution.fit(pool_train_solution)

In [90]:
pool_test_solution = Pool(test, cat_features = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'])

In [91]:
#Получение ответов

y_pred_solution = model.predict(pool_test_solution)

In [92]:
#Вот так они выглядят

y_pred_solution.astype(int)

array([1, 1, 0, ..., 1, 1, 0])

In [93]:
#Считывание данных в DataFrame 

test = pd.read_csv('/content/drive/MyDrive/Тула ИИ/test_dataset_test.csv', sep=';', index_col=None, dtype={'PATIENT_SEX':str, 'MKB_CODE':str, 'ADRES':str, 'VISIT_MONTH_YEAR':str, 'AGE_CATEGORY':str})

In [94]:
#Формируем sample_solution для отправки на платформу

test['PATIENT_ID_COUNT'] = y_pred_solution.astype(int)

In [95]:
#Сохраняем в csv файл
 
test.to_csv('sample_solution.csv', sep=';', index=None)